In [1]:
from IPython.core.events import EventManager
from IPython import get_ipython
import time

global_start_time = None

def start_timer(event):
    global global_start_time
    global_start_time = time.time()

def stop_timer(event):
    global global_start_time
    if global_start_time:
        duration = time.time() - global_start_time
        print(f"Cell execution time: {duration:.3f} seconds")
        global_start_time = None

ip = get_ipython()
ip.events.register('pre_run_cell', start_timer)
ip.events.register('post_run_cell', stop_timer)

In [2]:
from skimage import io 
import pandas as pd 
import matplotlib.pyplot as plt
import gc
import cv2
import numpy as np
from glob import glob 

Cell execution time: 6.424 seconds


In [3]:
df = pd.read_excel("../data/geomx_221_223.xlsx",sheet_name="SegmentProperties")
df = df.loc[df["SlideName"]=="1 PTB-22.1"]
df = df[["ROILabel","ROICoordinateX","ROICoordinateY","AOISurfaceArea"]]

Cell execution time: 1.024 seconds


In [4]:
xw = 512

Cell execution time: 0.000 seconds


In [5]:
# DAPI stain 
mat = io.imread("../channel/PTB22.1_Blue.tiff")
img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]

for x in img:
    np.save("../crop/221/"+"BLUE_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 31.828 seconds


In [6]:
# CD3 stain 
mat =  io.imread("../channel/PTB22.1_Green.tiff")

img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]


for x in img:
    np.save("../crop/221/"+"GREEN_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 41.409 seconds


In [7]:
# CD68 stain 
mat =  io.imread("../channel/PTB22.1_Red.tiff")

img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]


for x in img:
    np.save("../crop/221/"+"RED_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 38.600 seconds


In [8]:
# CD20 stain 
mat =  io.imread("../channel/PTB22.1_Yellow.tiff")

img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]

for x in img:
    np.save("../crop/221/"+"YELLOW_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 35.192 seconds


In [9]:
cpath = "../crop/221/"
imgs = glob(cpath+"*.npy")

roi = list(set([x.split("_")[-1].split(".npy")[0] for x in imgs]))
roi_dict = {int(r):[x for x in imgs if "_"+r+".npy" in x] for r in roi}

Cell execution time: 0.002 seconds


In [10]:
red_denom = 2**16
blue_denom = 2**12
green_denom = 2**16
yellow_denom = 2**12

blue_adjust = 0.7 
yellow_adjust = 0.7
red_adjust = 1.9 
green_adjust = 1.6

gamma = 1.2

for r in roi_dict:
    channels = roi_dict[r]
    imgB = np.load([x for x in channels if "BLUE" in x][0])
    imgR = np.load([x for x in channels if "RED" in x][0])
    imgG = np.load([x for x in channels if "GREEN" in x][0])
    imgY = np.load([x for x in channels if "YELLOW" in x][0])

    # scale and clip to a maximum pixel value 
    rawR = np.clip(imgR /     red_denom,    0, 1)
    rawG = np.clip(imgG /   green_denom,    0, 1)
    rawB = np.clip(imgB /    blue_denom,    0, 1)
    rawY = np.clip(imgY / yellow_denom,    0, 1)
    
    # image adjustment 
    imgR_norm = np.power(rawR, gamma) * red_adjust
    imgG_norm = np.power(rawG, gamma) * green_adjust
    imgB_norm = np.power(rawB, gamma) * blue_adjust
    imgY_norm = np.power(rawY, gamma) * yellow_adjust
    
    # red = red + yellow
    # green = green + yellow
    # blue = blue 
    imgR_final = np.clip(imgR_norm + imgY_norm, 0, 1)
    imgG_final = np.clip(imgG_norm + imgY_norm, 0, 1)
    imgB_final = np.clip(imgB_norm, 0, 1)

    img = np.stack((imgR_final, imgG_final, imgB_final), -1)
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(img)
    plt.axis("off")
    plt.savefig("../composite/"+"PTB221_"+str(r)+".png",dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

<Figure size 640x480 with 0 Axes>

Cell execution time: 73.742 seconds


In [11]:
df = pd.read_excel("../data/geomx_221_223.xlsx",sheet_name="SegmentProperties")
df = df.loc[df["SlideName"]=="2 PTB-22.3"]
df = df[["ROILabel","ROICoordinateX","ROICoordinateY","AOISurfaceArea"]]

Cell execution time: 0.952 seconds


In [12]:
# DAPI stain
mat = io.imread("../channel/PTB22.3_Blue.tiff")
img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]

for x in img:
    np.save("../crop/223/"+"BLUE_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 32.215 seconds


In [13]:
# CD3 stain
mat =  io.imread("../channel/PTB22.3_Green.tiff")

img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]


for x in img:
    np.save("../crop/223/"+"GREEN_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 34.510 seconds


In [14]:
# CD68 stain 
mat =  io.imread("../channel/PTB22.3_Red.tiff")

img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]


for x in img:
    np.save("../crop/223/"+"RED_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 38.536 seconds


In [15]:
# CD20 stain 
mat =  io.imread("../channel/PTB22.3_Yellow.tiff")

img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]

for x in img:
    np.save("../crop/223/"+"YELLOW_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 44.703 seconds


In [16]:
cpath = "../crop/223/"
imgs = glob(cpath+"*.npy")

roi = list(set([x.split("_")[-1].split(".npy")[0] for x in imgs]))
roi_dict = {int(r):[x for x in imgs if "_"+r+".npy" in x] for r in roi}

Cell execution time: 0.002 seconds


In [17]:
red_denom = 2**16
blue_denom = 2**12
green_denom = 2**16
yellow_denom = 2**12

blue_adjust = 0.7 
yellow_adjust = 0.7
red_adjust = 1.9 
green_adjust = 1.6

gamma = 1.2

for r in roi_dict:
    channels = roi_dict[r]
    imgB = np.load([x for x in channels if "BLUE" in x][0])
    imgR = np.load([x for x in channels if "RED" in x][0])
    imgG = np.load([x for x in channels if "GREEN" in x][0])
    imgY = np.load([x for x in channels if "YELLOW" in x][0])

    # scale and clip to a maximum pixel value 
    rawR = np.clip(imgR /     red_denom,    0, 1)
    rawG = np.clip(imgG /   green_denom,    0, 1)
    rawB = np.clip(imgB /    blue_denom,    0, 1)
    rawY = np.clip(imgY / yellow_denom,    0, 1)
    
    # image adjustment 
    imgR_norm = np.power(rawR, gamma) * red_adjust
    imgG_norm = np.power(rawG, gamma) * green_adjust
    imgB_norm = np.power(rawB, gamma) * blue_adjust
    imgY_norm = np.power(rawY, gamma) * yellow_adjust
    
    # red = red + yellow
    # green = green + yellow
    # blue = blue 
    imgR_final = np.clip(imgR_norm + imgY_norm, 0, 1)
    imgG_final = np.clip(imgG_norm + imgY_norm, 0, 1)
    imgB_final = np.clip(imgB_norm, 0, 1)

    img = np.stack((imgR_final, imgG_final, imgB_final), -1)

    fig,ax = plt.subplots(figsize=(16,16))
    plt.imshow(img)
    plt.axis("off")
    plt.savefig("../composite/"+"PTB223_"+str(r)+".png",dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

<Figure size 640x480 with 0 Axes>

Cell execution time: 42.559 seconds


In [18]:
df = pd.read_excel("../data/geomx_222.xlsx",sheet_name="SegmentProperties")
df = df.loc[df["SlideName"]=="PTB22.2"]
df = df[["ROILabel","ROICoordinateX","ROICoordinateY","AOISurfaceArea"]]

Cell execution time: 0.911 seconds


/Users/jperrie/miniconda3/envs/scvi/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [19]:
img_path = "../bb_annot/PTB222_1.png"
img=plt.imread(img_path)
xw = np.floor(img.shape[1]/2).astype(int)

Cell execution time: 0.042 seconds


In [20]:
# DAPI stain 
mat = io.imread("../channel/PTB22.2_Blue.tiff")
img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]

for x in img:
    np.save("../crop/222/"+"BLUE_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 38.084 seconds


In [21]:
# CD20 stain 
mat =  io.imread("../channel/PTB22.2_Green.tiff")

img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]


for x in img:
    np.save("../crop/222/"+"GREEN_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 38.450 seconds


In [22]:
# CD68 stain 
mat =  io.imread("../channel/PTB22.2_Red.tiff")

img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]


for x in img:
    np.save("../crop/222/"+"RED_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 33.871 seconds


In [23]:
# CD20 stain 
mat =  io.imread("../channel/PTB22.2_Yellow.tiff")

img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]

for x in img:
    np.save("../crop/222/"+"YELLOW_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 36.074 seconds


In [24]:
cpath = "../crop/222/"
imgs = glob(cpath+"*.npy")

roi = list(set([x.split("_")[-1].split(".npy")[0] for x in imgs]))
roi_dict = {int(r):[x for x in imgs if "_"+r+".npy" in x] for r in roi}

Cell execution time: 0.002 seconds


In [25]:
red_denom = 2**16
blue_denom = 2**14
green_denom = 2**15
yellow_denom = 2**13

blue_adjust = 0.9    
yellow_adjust = 0.8  
red_adjust = 1.3 
green_adjust = 1.6

gamma = 1.1

for r in roi_dict:
    channels = roi_dict[r]
    imgB = np.load([x for x in channels if "BLUE" in x][0])
    imgR = np.load([x for x in channels if "RED" in x][0])
    imgG = np.load([x for x in channels if "GREEN" in x][0])
    imgY = np.load([x for x in channels if "YELLOW" in x][0])

    # scale and clip to a maximum pixel value 
    rawR = np.clip(imgR /     red_denom,    0, 1)
    rawG = np.clip(imgG /   green_denom,    0, 1)
    rawB = np.clip(imgB /    blue_denom,    0, 1)
    rawY = np.clip(imgY / yellow_denom,    0, 1)
    
    # image adjustment 
    imgR_norm = np.power(rawR, gamma) * red_adjust
    imgG_norm = np.power(rawG, gamma) * green_adjust
    imgB_norm = np.power(rawB, gamma) * blue_adjust
    imgY_norm = np.power(rawY, gamma) * yellow_adjust
    
    # red = red + yellow
    # green = green + yellow
    # blue = blue 
    imgR_final = np.clip(imgR_norm + imgY_norm, 0, 1)
    imgG_final = np.clip(imgG_norm + imgY_norm, 0, 1)
    imgB_final = np.clip(imgB_norm, 0, 1)

    img = np.stack((imgR_final, imgG_final, imgB_final), -1)
    
    fig,ax = plt.subplots(figsize=(16,16))
    plt.imshow(img)
    plt.axis("off")
    plt.savefig("../composite/"+"PTB222_"+str(r)+".png",dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

<Figure size 640x480 with 0 Axes>

Cell execution time: 78.525 seconds


In [26]:
df = pd.read_excel("../data/geomx_211.xlsx",sheet_name="SegmentProperties")
df = df.loc[df["SlideName"]=="B172914-2"]
df = df[["ROILabel","ROICoordinateX","ROICoordinateY","AOISurfaceArea"]]

Cell execution time: 0.347 seconds


In [27]:
img_path = "../bb_annot/PTB211_1.png"
img=plt.imread(img_path)
xw = np.floor(img.shape[1]/2).astype(int)

Cell execution time: 0.132 seconds


In [28]:
# CD68 stain not DAPI 
mat = io.imread("../channel/PTB21.1_Blue.tiff")
img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]


for x in img:
    np.save("../crop/211/"+"BLUE_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 36.359 seconds


In [29]:
# DAPI stain not CD3 
mat =  io.imread("../channel/PTB21.1_Green.tiff")

img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]


for x in img:
    np.save("../crop/211/"+"GREEN_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 41.270 seconds


In [30]:
# CD8 stain not CD68
mat =  io.imread("../channel/PTB21.1_Red.tiff")

img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]


for x in img:
    np.save("../crop/211/"+"RED_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 33.966 seconds


In [31]:
# CD3 stain not CD20 
mat =  io.imread("../channel/PTB21.1_Yellow.tiff")

img={}
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=int(x)
    y=int(y)
    img[roi]=mat[(x-xw):(x+xw),(y-xw):(y+xw)]

for x in img:
    np.save("../crop/211/"+"YELLOW_"+str(x)+".npy",img[x])

del img
del mat

Cell execution time: 36.998 seconds


In [32]:
cpath = "../crop/211/"
imgs = glob(cpath+"*.npy")

roi = list(set([x.split("_")[-1].split(".npy")[0] for x in imgs]))
roi_dict = {int(r):[x for x in imgs if "_"+r+".npy" in x] for r in roi}

Cell execution time: 0.001 seconds


In [33]:
red_denom = 2**13
blue_denom = 2**14
green_denom = 2**12
yellow_denom = 2**13

blue_adjust = 1.2
yellow_adjust = 1.1 
red_adjust = 1.8
green_adjust = 1.8

gamma = 1.4

for r in roi_dict:
    channels = roi_dict[r]
    imgY = np.load([x for x in channels if "BLUE" in x][0]) 
    imgR = np.load([x for x in channels if "RED" in x][0])
    imgB = np.load([x for x in channels if "GREEN" in x][0])
    imgG = np.load([x for x in channels if "YELLOW" in x][0])

    # scale and clip to a maximum pixel value 
    rawR = np.clip(imgR /     red_denom,    0, 1)
    rawG = np.clip(imgG /   green_denom,    0, 1)
    rawB = np.clip(imgB /    blue_denom,    0, 1)
    rawY = np.clip(imgY / yellow_denom,    0, 1)
    
    # image adjustment 
    imgR_norm = np.power(rawR, gamma) * red_adjust
    imgG_norm = np.power(rawG, gamma) * green_adjust
    imgB_norm = np.power(rawB, gamma) * blue_adjust
    imgY_norm = np.power(rawY, gamma) * yellow_adjust
    
    # red = red
    # green = green + yellow
    # blue = blue + yellow 
    imgR_final = np.clip(imgR_norm,                0, 1)
    imgG_final = np.clip(imgG_norm + imgY_norm,    0, 1)
    imgB_final = np.clip(imgB_norm + imgY_norm,    0, 1)
    
    img = np.stack((imgR_final, imgG_final, imgB_final), axis=-1)

    fig,ax = plt.subplots(figsize=(16,16))
    plt.imshow(img)
    plt.axis("off")
    plt.savefig("../composite/"+"PTB211_"+str(r)+".png",dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()
    

<Figure size 640x480 with 0 Axes>

Cell execution time: 128.224 seconds
